Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from itertools import product
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse
from functools import reduce
from copy import copy
from collections import deque

In [2]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [16]:
NUM_POINTS = 5000
NUM_SETS = NUM_POINTS
DENSITY = .3


In [17]:
x = make_set_covering_problem(NUM_POINTS, NUM_SETS, DENSITY)
# print("Element at row=42 and column=42:", x[42, 42])

In [18]:
SETS = x.toarray()

In [7]:
current_state = [choice([False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False,  True  , False, False, False, False, False, False, False, False,False, False, False, False, False, False, False, False,False, False, False, False, False, False, False, False,False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False,False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]) for _ in range(NUM_SETS)]
print("current state", current_state, fitness(current_state), sep="\n")

print(reduce(np.logical_and, reduce(np.logical_or,
       [SETS[i] for i, t in enumerate(current_state) if t],
       np.array([False for _ in range(NUM_POINTS)])
       ), True))

NameError: name 'fitness' is not defined

In [10]:
num_covered_critic_point = min(sum(SETS))
critic_point = np.argmin(sum(SETS))
state_that_cover_critic_point = [index for index, val in enumerate(SETS) if val[critic_point]]
count=0
print(num_covered_critic_point, critic_point, state_that_cover_critic_point)

state_that_cover_critic_point_sorted = sorted(state_that_cover_critic_point, key=lambda idx: sum(SETS[idx]), reverse=True)
print(state_that_cover_critic_point_sorted)


649 860 [0, 1, 6, 8, 10, 11, 12, 13, 14, 15, 17, 18, 19, 24, 26, 27, 28, 29, 30, 31, 32, 34, 36, 37, 40, 42, 43, 46, 47, 49, 50, 51, 53, 54, 58, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 78, 79, 80, 82, 83, 84, 85, 86, 87, 92, 93, 94, 95, 97, 98, 99, 100, 101, 103, 105, 108, 110, 111, 112, 113, 115, 116, 117, 118, 119, 120, 121, 123, 125, 127, 128, 129, 130, 133, 137, 138, 141, 142, 144, 146, 147, 149, 150, 151, 153, 154, 156, 157, 159, 161, 163, 164, 165, 169, 173, 174, 175, 176, 177, 178, 179, 181, 183, 185, 187, 189, 190, 191, 192, 193, 194, 195, 196, 198, 199, 200, 201, 202, 204, 206, 207, 210, 211, 212, 217, 218, 219, 220, 223, 225, 226, 228, 230, 231, 234, 235, 236, 237, 238, 239, 242, 243, 245, 246, 247, 249, 250, 251, 254, 255, 256, 258, 259, 260, 261, 262, 267, 268, 269, 270, 273, 274, 276, 278, 280, 281, 282, 284, 286, 287, 289, 290, 291, 292, 293, 296, 298, 299, 300, 301, 302, 303, 304, 305, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 320, 321, 322, 3

In [19]:
COUNT_FITNESS = 0
def fitness(state):
    global COUNT_FITNESS
    COUNT_FITNESS += 1
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost

In [21]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, NUM_SETS - 1)
    new_state[index] = not new_state[index]
    return new_state

def tweak2(state):
    new_state = copy(state)
    num_state_to_change = randint(1, sum(state))
    state_to_change = [index for index, value in enumerate(state) if value]
    
    index = choice(state_to_change)
    state_to_change.remove(index)
    new_state[index] = not new_state[index]

    for _ in range(num_state_to_change-1):
        index = choice(state_to_change)
        state_to_change.remove(index)
        new_state[index] = not new_state[index]

        new_index = randint(0, NUM_SETS - 1)
        new_state[new_index] = not new_state[new_index]
        
    return new_state

def perturb(state, round):
    new_state = copy(state)
    num_state_to_change = randint(1, sum(state))
    state_to_change = [index for index, value in enumerate(state) if value]
    
    index = choice(state_to_change)
    state_to_change.remove(index)
    new_state[index] = not new_state[index]

    for _ in range(num_state_to_change-round):
        index = choice(state_to_change)
        state_to_change.remove(index)
        new_state[index] = not new_state[index]

        new_index = randint(0, NUM_SETS - 1)
        new_state[new_index] = not new_state[new_index]
        
    return new_state

## Halloween challenge - Prof version

In [ ]:
current_state = [choice([False, False, False, False, False, False, False, False, False, False]) for _ in range(NUM_SETS)]
#print("SETS", SETS, current_state, fitness(current_state), sep="\n")
count = 0
for step in range(int( NUM_POINTS/(DENSITY*100) )):
    new_state = tweak(current_state)
    count += 2
    if fitness(new_state) >= fitness(current_state):
        current_state = new_state
        print(fitness(current_state))
        count += 1

## Halloween Challenge - New Tweak + with different approach

In [219]:
current_state = [choice([False, False, False, False, False, False, False, False, False, False]) for _ in range(NUM_SETS)]
#print("SETS", SETS, current_state, fitness(current_state), sep="\n")
count = 0
for step in range(int( NUM_POINTS/(DENSITY*100) )):       #-> THIS IS THE MAIN PROBLEM !!! -> HAVE TO BE AUTOMATIC
    #print(step)
    current_fitness = fitness(current_state)
    count += 1
    if current_fitness[0] == NUM_POINTS:
        new_state = tweak2(current_state)
        new_fit = fitness(new_state)
        count += 1
        if new_fit > current_fitness:
            current_state = new_state
            print(fitness(current_state))
            count += 1
    else:
        new_state = tweak(current_state)
        current_state = new_state
    #print("current fitness->",current_fitness)

(5000, -29)
(5000, -28)
(5000, -27)
(5000, -26)
(5000, -25)
(5000, -24)
(5000, -23)
(5000, -22)


## Halloween Challenge - Hard slection as starting point the sets that cover the lowest covered point (crescent order)

In [ ]:
current_state = [choice([False, False, False, False, False, False, False, False, False, False]) for _ in range(NUM_SETS)]
best_state = []
best_fitness = (NUM_POINTS, -NUM_SETS)
new_state = []

# Define the number of starting point -> Crtitic !!
num_covered_critic_point = min(sum(SETS))   # Num of sub-sets that cover the less covered point -> point is easier to be critic!!
critic_point = np.argmin(sum(SETS))     # The critic point
state_that_cover_critic_point = [index for index, val in enumerate(SETS) if val[critic_point]]      # All the sub-sets that cover the critic_point
state_that_cover_critic_point_sorted = sorted(state_that_cover_critic_point, key=lambda idx: sum(SETS[idx]), reverse=True)
num_sub_sets_for_coverage = int( NUM_POINTS/(DENSITY*100) )    #Starting, to cover all (sub-optima for sure)


for i in state_that_cover_critic_point:     #Start the examinate with the set that cover a critic point
    current_state[i] = not current_state[i]
    step = 3

    for t_max in range(num_sub_sets_for_coverage):      # Starting num of set needed
        new_state = tweak(current_state)
        current_state = new_state 
        # START from here !!

    current_fitness = fitness(current_state)
    print(current_fitness)

    while step != 0 and current_fitness[0]==NUM_POINTS:        # I try adaptation step after I reach the local maxima

        if current_fitness[0] == NUM_POINTS:
            step -= 1
            new_state = tweak2(current_state)
            new_fitness = fitness(new_state)
            if new_fitness > current_fitness:
                print(new_fitness)
        else:
            new_state = tweak(current_state)
            new_fitness = fitness(new_state)
            current_state = new_state

        current_state = new_state
        current_fitness = new_fitness
    
    if current_fitness > best_fitness:
        best_state = current_state
        best_fitness = current_fitness
    num_sub_sets_for_coverage = np.abs(best_fitness[1])
    current_state = [choice([False]) for _ in range(NUM_SETS)]


## Halloween Clallenge - Just Perturb a solution that is considered local optima for at max 5 iteration

In [22]:
current_state = [choice([False, False, False, False, False, False, False, False, False, False]) for _ in range(NUM_SETS)]
new_state =[]
best_state = []
best_fitness = (NUM_POINTS, -NUM_SETS)

count = 0

for step in range(int( NUM_POINTS/(DENSITY*100) )):
    round = 5
    current_fitness = fitness(current_state)
    while round != 0:
        if current_fitness[0] == NUM_POINTS:
            new_state = perturb(current_state, round)
            new_fitness = fitness(new_state)
            if new_fitness > current_fitness:
                print(new_fitness)
                current_state = new_state
                current_fitness = new_fitness
            round -= 1
        else:
            new_state = tweak(current_state)
            new_fitness = fitness(new_state)
            current_state = new_state
            current_fitness = new_fitness
    
    best_fitness = current_fitness
    best_state = current_state


(5000, -27)
(5000, -26)
(5000, -25)
(5000, -24)
(5000, -23)
(5000, -22)
(5000, -21)
(5000, -20)
(5000, -19)


## Retain the result

In [ ]:
best_state

In [23]:
best_fitness

(5000, -19)

In [24]:
COUNT_FITNESS

1026